In [1]:
%pip install -q -U kaggle_environments

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

1 Опишем поведение агента, всегда играющего 'камень' - это значение 0

In [2]:
%%writefile rock_agent.py

# Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def only_rock(observation, configuration):
    return 0

Overwriting rock_agent.py


2 Опишем поведение агента, всегда играющего 'бумага' - это значение 1

In [3]:
%%writefile paper_agent.py

def only_paper(observation, configuration):
    return 1

Overwriting paper_agent.py


3 Опишем поведение агента, всегда играющего 'ножницы' - это значение 2

In [4]:
%%writefile scissors_agent.py

def only_scissors(observation, configuration):
    return 2

Overwriting scissors_agent.py


4 Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [5]:
%%writefile copy_opponent.py
import random

# Example
def copy_opponent(observation, configuration):
    # In case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    # Initial step
    else:
        return random.randint(0, 2)

Overwriting copy_opponent.py


5 Опишем агента, который всегда производит победное действие над прошлым ходом оппонента.

In [6]:
%%writefile late_champion.py
import random

# Example
def late_champion(observation, configuration):
    # In case we have information about opponent last move
    if observation.step > 0:
        a = observation.lastOpponentAction
        if a == 2:
            return a - 2
        else:
            return a + 1
    # Initial step
    else:
        return random.randint(0, 2)

Overwriting late_champion.py


6 Опишем агента, который всегда делает случайный ход

In [7]:
%%writefile random_action.py
import random

def random_action(observation, configuration):
        return random.randint(0, 2)

Overwriting random_action.py


7 Опишем агента, который всегда выбирает "камень" или "бумагу".

In [8]:
%%writefile rock_or_paper.py
import random

def rock_or_paper(observation, configuration):
    return random.choice([0, 1])

Overwriting rock_or_paper.py


8 Опишем агента, который всегда выбирает "камень" или "ножницы".

In [9]:
%%writefile rock_or_scissors.py
import random

def rock_or_scissors(observation, configuration):
    return random.choice([0, 2])

Overwriting rock_or_scissors.py


9 Опишем агента, который всегда выбирает "бумагу" или "ножницы".

In [11]:
%%writefile paper_or_scissors.py
import random

def paper_or_scissors(observation, configuration):
    return random.choice([1, 2])

Overwriting paper_or_scissors.py


10 Опишем агента, который выбирает победный (или беспроигрышный) вариант над самым редким ходом противника.

In [53]:
%%writefile champion_of_rare.py
import random

rock = 0
paper = 0
scissors = 0

def champion_of_rare(observation, configuration):
    global rock, paper, scissors
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            rock += 1
        elif observation.lastOpponentAction == 1:
            paper += 1
        else:
            scissors += 1
        if (rock < paper) and (rock < scissors):
            return 1
        elif (paper < scissors) and (paper < rock):
            return 2
        elif (scissors < rock) and (scissors < paper):
            return 0            
        elif (rock == paper) and (rock < scissors):
            return 1
        elif (rock == scissors) and (rock < paper):
            return 0
        elif (paper == scissors) and (paper < rock):
            return 2
    return random.randint(0, 2)

Overwriting champion_of_rare.py


11 Опишем агента, который выбирает победный (или беспроигрышный) вариант над самым частым ходом противника.

In [82]:
%%writefile champion_of_freq.py
import random

rock = 0
paper = 0
scissors = 0

def champion_of_freq(observation, configuration):
    global rock, paper, scissors
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            rock += 1
        elif observation.lastOpponentAction == 1:
            paper += 1
        else:
            scissors += 1
        if (rock > paper) and (rock > scissors):
            return 1
        elif (paper > scissors) and (paper > rock):
            return 2
        elif (scissors > rock) and (scissors > paper):
            return 0            
        elif (rock == paper) and (rock > scissors):
            return 1
        elif (rock == scissors) and (rock > paper):
            return 0
        elif (paper == scissors) and (paper > rock):
            return 2
    return random.randint(0, 2)

Writing champion_of_freq.py


12 Опишем агента, который выбирает действие по очереди.

In [182]:
%%writefile agent_line.py

list_now = [1, 2, 3]

def agent_queue(observation, configuration):
    global list_now
    if list_now == [0, ]:
        list_now.append(1)
        return 1
    elif list_now == [0, 1]:
        list_now.append(2)
        return 2
    list_now = [0]
    return 0

Writing agent_line.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [123]:
evaluate(
    'rps', # Environment to use - no need to change
    ['champion_of_freq.py', 'rock_agent.py'], # Agents to evaluate
    configuration={'episodeSteps': 100, 'tieRewardThreshold': 1} # Number of episodes
)

[[98.0, -98.0]]

In [37]:
evaluate(
    'rps', # Environment to use - no need to change
    ['late_champion.py', 'random_action.py'], # Agents to evaluate
    configuration={'episodeSteps': 100, 'tieRewardThreshold': 1} # Number of episodes
)

[[2.0, -2.0]]

In [155]:
evaluate(
    'rps', # Environment to use - no need to change
    ['scissors_agent.py', 'agent_queue'], # Agents to evaluate
    configuration={'episodeSteps': 13, "tieRewardThreshold": 1} # Number of episodes
)

[[0, 0]]

In [65]:
evaluate(
    'rps', # Environment to use - no need to change
    ['paper_or_scissors.py', 'random_action.py'], # Agents to evaluate
    configuration={'episodeSteps': 50, "tieRewardThreshold": 10} # Number of episodes
)

[[0, 0]]

### ТУРНИР

Команды играют в 2 круга. В каждой встрече 100 раундов. За победу агент получает 3 очка, за ничью - 1. Победителем турнира является тот, у кого больше очков.

In [189]:
agents_dict= {
    'rock_agent': 'rock_agent.py',
    'paper_agent': 'paper_agent.py',
    'scissors_agent': 'scissors_agent.py',
    'copy_opponent': 'copy_opponent.py',
    'late_champion': 'late_champion.py',
    'random_action': 'random_action.py',
    'rock_or_paper': 'rock_or_paper.py',
    'rock_or_scissors': 'rock_or_scissors.py',
    'paper_or_scissors': 'paper_or_scissors.py', 
    'champion_of_rare': 'champion_of_rare.py',
    'champion_of_freq': 'champion_of_freq.py',
    'agent_line': 'agent_line.py'
}

standings = {}
for agent_i in agents_dict:
    standings[agent_i] = 0

for left in agents_dict.keys():
    for right in agents_dict.keys():
        if left != right:
            result = evaluate(
                'rps',
                [agents_dict[left], agents_dict[right]],
                configuration={'episodeSteps': 100, "tieRewardThreshold": 1}
            )
            if result[0][0] > result[0][1]:
                standings[left] += 3
            elif result[0][0] == result[0][1]:
                standings[left] += 1
                standings[left] += 1
            else:
                standings[right] += 3

print(standings)

{'rock_agent': 32, 'paper_agent': 32, 'scissors_agent': 29, 'copy_opponent': 22, 'late_champion': 53, 'random_action': 30, 'rock_or_paper': 33, 'rock_or_scissors': 29, 'paper_or_scissors': 26, 'champion_of_rare': 15, 'champion_of_freq': 51, 'agent_line': 33}


In [197]:

print('_________________________')

max_points = 0
for agent, points in standings.items():
    print(agent, points)
    if points > max_points:
        max_points = points
        champion = []
        champion.append(agent)

print('_________________________')

if len(champion) == 1:
    print('Победитель турнира:', champion[0])
else:
    print('Первое место делят агенты:', champion)

_________________________
rock_agent 32
paper_agent 32
scissors_agent 29
copy_opponent 22
late_champion 53
random_action 30
rock_or_paper 33
rock_or_scissors 29
paper_or_scissors 26
champion_of_rare 15
champion_of_freq 51
agent_line 33
_________________________
Победитель турнира: late_champion
